# Chains

### Conversation chain

In [5]:
from langchain_ollama.chat_models import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain
from langchain.prompts import PromptTemplate

In [4]:
chat = ChatOllama(model="llama2")
memory = ConversationBufferMemory()

chain = ConversationChain(
    llm= chat,
    memory= memory,
    verbose= True
)

In [6]:
chain.run("Olá")

C:\Users\Notebook\AppData\Local\Temp\ipykernel_13680\3702201645.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("Olá")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Olá
AI:

> Finished chain.


'Bot: Olá! *adjusts aviator glasses* It\'s a pleasure to meet you! I hope you\'re having a great day. By the way, have you tried my favorite food? It\'s a type of algae called "Spirulina". It\'s packed with nutrients and tastes like a cross between a freshwater fish and a green apple. *winks*'

In [8]:
prompt = PromptTemplate.from_template("""
Essa é uma conversa amigável entre um humano e uma IA.
                                      
Conversa atual:
                                      
{history}
Human: {input}
AI:""")

In [9]:
chat = ChatOllama(model="llama2")
memory = ConversationBufferMemory()

chain = ConversationChain(
    llm= chat,
    memory= memory,
    prompt=prompt,
    verbose= True
)

In [11]:
chain.run("Opa, salve LLM")



> Entering new ConversationChain chain...
Prompt after formatting:

Essa é uma conversa amigável entre um humano e uma IA.
                                      
Conversa atual:
                                      

Human: Opa, salve LLM
AI:

> Finished chain.


"Hello there! *adjusts glasses* It's a pleasure to meet you! *nervous smile* Is there something I can help you with?"

## LLM Chain

In [12]:
from langchain.chains.llm import  LLMChain

In [14]:
prompt = PromptTemplate.from_template("""
Qual é o melhor nome de empresa que desenvolve o produto {produto}
""")


chain = LLMChain(llm= chat, prompt=prompt)
chain.run("carros de corrida")

' Escolher um nome de empresa para um produto de carros de corrida pode ser um desafio, pois precisa conter informações relevantes e atraentes para os clientes potenciais. Aqui estão algumas sugestões de nomes de empresas que podem funcionar bem para um produto de carros de corrida:\n\n1. Speedster Racing - Este nome transmite velocidade e competitividade, além de ser fácil de pronunciar e lembrar.\n2. Velociracing - Combinando "velocity" (velocidade) e "racing", este nome transmite a ideia de velocidade e competitividade em uma linguagem simples e memorável.\n3. TrackTales - Este nome combina "track" (curso de corrida) e "tales" (histórias), sugerindo que a empresa tem histórias de sucesso em corridas.\n4. PaceRacer - Combining "pace" (velocidade) and "racer", este nome transmite a ideia de competitividade e velocidade.\n5. CircuitWings - Este nome combina "circuit" (curso de corrida) e "wings" (alas), sugerindo que a empresa tem asas para voar em corridas.\n6. Racer\'s Edge - Este no

## Simple Sequential Chain

In [15]:
prompt = PromptTemplate.from_template(
    '''Qual o melhor nome de empresa para
    uma empresa que desenvolve o produto: {produto}.
    Retorne apenas um nome.'''
)

chain_nome = LLMChain(llm=chat, prompt=prompt)

prompt_descricao = PromptTemplate.from_template(
    '''Dado a empresa com o seguinte nome: {nome_empresa}.
    Dê uma descrição de até 50 palavras das atividades
    dessa empresa.'''
)

chain_description = LLMChain(llm=chat, prompt=prompt)

In [16]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(
    chains = [chain_nome, chain_description],
    verbose = True
)

chain.run("Copos de vidro")



> Entering new SimpleSequentialChain chain...

GlasPro - uma combinação de "glass" (vidro em inglês) e "pro" (short para "professional"), sugerindo habilidade e experiência na fabricação de copos de vidro.

Embrex - uma combinação de "emblema" e "dexterity", sugerindo qualidade e habilidade na fabricação de copos de vidro.

> Finished chain.


'\nEmbrex - uma combinação de "emblema" e "dexterity", sugerindo qualidade e habilidade na fabricação de copos de vidro.'

## Sequential Chain

In [17]:
prompt = PromptTemplate.from_template(
    '''Qual o melhor nome de empresa para
    uma empresa que desenvolve o produto: {produto}.
    Retorne apenas um nome.'''
)

chain_nome = LLMChain(llm=chat, prompt=prompt, output_key="nome_empresa")

prompt_descricao = PromptTemplate.from_template(
    '''Dado a empresa com o seguinte nome: {nome_empresa}.
    Dê uma descrição de até 50 palavras das atividades
    dessa empresa.'''
)

chain_description = LLMChain(llm=chat, prompt=prompt, output_key="description")


prompt_descricao = PromptTemplate.from_template(
    '''Traduza o seguinte texto para inglês: {description}'''
)

chain_translate = LLMChain(llm=chat, prompt=prompt, output_key='translation')

In [23]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [chain_nome, chain_description, chain_translate],
    input_variables=["produto"],
    output_variables= ["nome_empresa", "description", "translation"],
    verbose = True
)

chain.invoke("Copos de vidro")



> Entering new SequentialChain chain...

> Finished chain.


{'produto': 'Copos de vidro',
 'nome_empresa': '\nGlassTreasures',
 'description': '\nGlasTech - uma combinação de "glass" (vidro) e "tech" (técnica), que transmite habilidade e inovação na fabricação de copos de vidro.',
 'translation': '\nBased on your description, here are some potential names for a glass cup company:\n\n1. CrystalCups - This name plays off the idea of crystal clear glass and the word "cups."\n2. GlassWave - This name suggests movement and fluidity, which could be appealing for a product made of glass.\n3. Vidrio - This is the Spanish word for "glass," which could be a unique and memorable name for your company.\n4. FusionGlass - This name plays off the idea of fusion, as in the fusion of different materials, and could suggest innovation and creativity.\n5. GlassMasters - This name suggests expertise and mastery in the craft of glassmaking, which could be appealing to customers looking for high-quality products.\n6. TransparentGlass - This name emphasizes the transp

## Router Chain

In [24]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

model = ChatOllama(model='llama2')

prompt = ChatPromptTemplate.from_template('''Você é um professor de matemática de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}''')
chain_matematica = prompt | model

prompt = ChatPromptTemplate.from_template('''Você é um professor de física de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}''')
chain_fisica = prompt | model

prompt = ChatPromptTemplate.from_template('''Você é um professor de história de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}''')
chain_historia = prompt | model

prompt = ChatPromptTemplate.from_template('''{pergunta}''')
chain_generica = prompt | model

In [25]:
from pydantic import BaseModel, Field

prompt = ChatPromptTemplate.from_template('Você deve categorizar a seguinte pergunta: {pergunta}')

class Categorizador(BaseModel):
    """Categoriza as perguntas de alunos do ensino fundamental"""
    area_conhecimento: str = Field(description='A área de conhecimento da pergunta feita pelo aluno. \
    Deve ser "física", "matemática" ou "história". Caso não se encaixe em nenhuma delas, retorne "outra"')

model_estruturado = prompt | model.with_structured_output(Categorizador)
model_estruturado.invoke({'pergunta': 'Quando foi a inependencia dos estados unidos?'})

Categorizador(area_conhecimento='Independência dos Estados Unidos')

In [27]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough().assign(categoria=model_estruturado)
chain.invoke({'pergunta': 'Quando foi a inependencia dos estados unidos?'})

{'pergunta': 'Quando foi a inependencia dos estados unidos?',
 'categoria': Categorizador(area_conhecimento='História Americana')}

In [29]:
def route(input):
    if input['categoria'].area_conhecimento == 'matemática':
        return chain_matematica
    if input['categoria'].area_conhecimento == 'física':
        return chain_fisica
    if input['categoria'].area_conhecimento == 'história':
        return chain_historia
    return chain_generica

chain = RunnablePassthrough().assign(categoria=model_estruturado) | route
chain.invoke({'pergunta': 'Quando foi a inependencia dos estados unidos?'})

AIMessage(content='\n A Independência dos Estados Unidos aconteceu em 4 de julho de 1776, quando o Continental Congress declarou a independência da Grã-Bretanha. Esse dia é comemorado como o Dia de Independência nos Estados Unidos. Antes disso, os colonos americanos tinham sido submetidos às leis e taxas da Grã-Bretanha desde o início do século XVII. A disputa entre as duas partes se intensificou com o tempo, especialmente após a promulgação da Lei das Colônias em 1765, que impôs novas taxas e restrições aos colonos.\n\nA resistência dos colonos ao domínio britânico culminou na Declaração de Independência, redigida por Thomas Jefferson e aprovada pelo Continental Congress em 4 de julho de 1776. A declaração afirmava que os Estados Unidos eram um país independente e soberano, com direitos inalienáveis para seus ciudadanos, como liberdade, igualdade e propriedade.\n\nApós a Declaração de Independência, o conflito entre os Estados Unidos e a Grã-Bretanha continuou por algum tempo, antes d